## Analysis Extended Measurement Range

In [ ]:
# SETTINGS AND CONFIGURATION

# Set basefolder to the data
base_folder = ''

# set stimulus window offsets
PRE_OFFSET_TIME = 0.5
POST_OFFSET_TIME = 1.5

# Camera and light source colors for timeline plots
CAMERA_COLORS = {
    'cam_0': '#1f77b4',  # blue
    'cam_1': '#ff7f0e',  # orange
    'cam_2': '#2ca02c',  # green
    'cam_3': '#d62728',  # red
    'cam_4': '#9467bd',  # purple
}

LIGHT_SOURCE_COLORS = {
    'light_0': '#e377c2',  # pink
    'light_1': '#8c564b',  # brown
    'light_2': '#7f7f7f',  # gray
    'light_3': '#bcbd22',  # olive
    'light_4': '#17becf',  # cyan
}

In [ ]:
# import gaze data for extended measurement range experiment for all subjects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import scipy

import sys
import os


# Add the software folder to the path (parent.parent of this notebook)
notebook_dir = Path(os.getcwd())  # or use Path.cwd()
project_root = notebook_dir.parent.parent  # Goes up to /home/arthur/Projects/AREA
sys.path.insert(0, str(project_root))

# Define subjects and base path
subjects = [
    'emr01',
    'emr02',
    'emr03',
    'emr04',
    'emr05',
    'emr06'
        ]

# Load data for all subjects
all_data = []

for subject_id in subjects:
    data_path = Path(base_folder) / subject_id / 'pre_analysis_results' / 'measurement'
    gaze_file = data_path / 'gaze_estimation_results.csv'
    
    if gaze_file.exists():
        df = pd.read_csv(gaze_file)
        df['subject_id'] = subject_id
        all_data.append(df)
        print(f"✓ Loaded {len(df)} frames for {subject_id}")
    else:
        print(f"✗ File not found for {subject_id}: {gaze_file}")

# Combine all data
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"\nTotal frames loaded: {len(combined_df)}")
    print(f"Subjects: {', '.join(combined_df['subject_id'].unique())}")
else:
    print("No data loaded!")
    combined_df = pd.DataFrame()


In [ ]:
# Set publication-quality plot settings
plt.rcParams.update({
    # Font sizes
    'font.size': 10,           # Base font size
    'axes.titlesize': 12,      # Subplot titles
    'axes.labelsize': 11,      # Axis labels
    'xtick.labelsize': 9,      # X-axis tick labels
    'ytick.labelsize': 9,      # Y-axis tick labels
    'legend.fontsize': 9,      # Legend
    'figure.titlesize': 14,    # Figure title (suptitle)
    
    # Figure quality
    'figure.dpi': 100,         # Screen display
    'savefig.dpi': 300,        # Saved figure quality
    'savefig.bbox': 'tight',   # Tight bounding box
    
    # Font properties
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'Helvetica', 'DejaVu Sans'],
    
    # Line widths
    'axes.linewidth': 0.8,
    'grid.linewidth': 0.5,
    'lines.linewidth': 1.5,
})

sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.0)

print("✓ Publication-quality plot settings configured")
print("  - Font sizes: Title=12pt, Labels=11pt, Ticks=9pt, Legend=9pt")
print("  - Save resolution: 300 DPI")
print("  - Style: Seaborn whitegrid with paper context")

In [ ]:
# Create output folder for figures
output_folder = Path('figures')
output_folder.mkdir(exist_ok=True)
print(f"Figures will be saved to: {output_folder.absolute()}")

# Simulation

In [ ]:
# Load .mat file
simulation_file = Path(base_folder) / 'simulation' / 'detectCount_allLights.mat'
mat = scipy.io.loadmat(simulation_file)
detectCount = mat['detectCount']  # nH x nCams x nLights
HoriEye = mat['HoriEye'][0]
HposCam = mat['HposCam'][0]
Li = mat['Li']  # 3 x nLights

order = np.argsort(Li[0, :])   # sorting indices based on first row
Li = Li[:, order]   

Li_neg = Li[:, Li[1, :] < 0]
Li_pos = Li[:, Li[1, :] >= 0]

# Li_neg = Li_neg[:,2:5]
# Li_pos = Li_pos[:,2:5]

nLights = detectCount.shape[2]

# Determine subplot layout (similar to MATLAB tiledlayout)
nCols = 2
nRows = Li_neg.shape[1]

fig, axes = plt.subplots(nRows, nCols, figsize=(4*nCols, 3*nRows))

for L in range(Li_neg.shape[1]):
    # Create DataFrame for Seaborn
    df = pd.DataFrame(detectCount[:,:,L], index=HoriEye, columns=HposCam)
    df = df.iloc[::-1]  # flip vertically to match MATLAB orientation
    
    sns.heatmap(df, ax=axes[L,0], cmap='RdYlGn', cbar=True, vmin=0, vmax=df.values.max())
    axes[L,0].set_xlabel('Camera X (mm)')
    axes[L,0].set_title(f'Light X={Li_neg[0,L]:.1f} mm, Y={Li_neg[1,L]:.1f} mm')
    axes[L,0].set_ylabel('Horizontal Eye Orientation ($^\circ$)')

for L in range(Li_pos.shape[1]):
    # Create DataFrame for Seaborn
    idx = L + Li_neg.shape[1]  # Correct index into detectCount
    df = pd.DataFrame(detectCount[:,:,idx], index=HoriEye, columns=HposCam)
    df = df.iloc[::-1]  # flip vertically to match MATLAB orientation
    
    sns.heatmap(df, ax=axes[L,1], cmap='RdYlGn', cbar=True, vmin=0, vmax=df.values.max())
    axes[L,1].set_xlabel('Camera X (mm)')
    axes[L,1].set_title(f'Light X={Li_pos[0,L]:.1f} mm, Y={Li_pos[1,L]:.1f} mm')
    axes[L,1].set_ylabel('Horizontal Eye Orientation ($^\circ$)')

# Hide unused subplots
for i in range(max(Li_neg.shape[1], Li_pos.shape[1]), nRows):
    axes[i,0].axis('off')
    axes[i,1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Round fixation positions to 3 decimal places
combined_df['fixation_positionX_deg'] = combined_df['fixation_positionX_deg'].round(decimals=2)
combined_df['fixation_positionY_deg'] = combined_df['fixation_positionY_deg'].round(decimals=2)

In [ ]:
# Only include frames where stimulus was present

# Filter to only frames with valid stimulus data
valid_frames = combined_df[
    combined_df['fixation_positionX_deg'].notna() & 
    combined_df['fixation_positionY_deg'].notna()
].copy()

print(f"Valid frames with stimulus data: {len(valid_frames)}")
print(f"Percentage of frames with fixation data: {100*len(valid_frames)/len(combined_df):.1f}%")


# Outlier Analysis

In [ ]:
# Analyze outlier percentage relative to total fixations

# Filter to only frames with valid stimulus data (same as earlier)
frames_with_stimulus = combined_df[
    combined_df['fixation_positionX_deg'].notna() & 
    combined_df['fixation_positionY_deg'].notna()
].copy()

# Calculate overall outlier statistics for frames
total_frames_with_stimulus = len(frames_with_stimulus)
outlier_frames = len(frames_with_stimulus[frames_with_stimulus['is_outlier'] == 1])
valid_frames_count = len(frames_with_stimulus[frames_with_stimulus['is_outlier'] == 0])
outlier_percentage = (outlier_frames / total_frames_with_stimulus) * 100

# Calculate fixation window statistics
# Count unique fixation windows (stimulus presentations) per subject and eye
total_fixation_windows = 0
outlier_fixation_windows = 0

for subject_id in frames_with_stimulus['subject_id'].unique():
    subject_data = frames_with_stimulus[frames_with_stimulus['subject_id'] == subject_id]
    
    for eye in ['left', 'right']:
        eye_data = subject_data[subject_data['eye'] == eye]
        
        # Get all unique stimulus IDs
        unique_stimuli = eye_data['stimulus_id'].dropna().unique()
        
        for stim_id in unique_stimuli:
            # Get all frames for this stimulus
            stim_frames = eye_data[eye_data['stimulus_id'] == stim_id].sort_values('timestamp')
            
            if len(stim_frames) > 0:
                total_fixation_windows += 1
                
                # Find the first timestamp for this stimulus
                first_timestamp = stim_frames['timestamp'].iloc[0]
                
                # Define the time window: 0.5s to 1.5s after first occurrence
                window_start = first_timestamp + PRE_OFFSET_TIME
                window_end = first_timestamp + 1 + POST_OFFSET_TIME

                # Get frames within this time window
                filtered_stim_frames = stim_frames[
                    (stim_frames['timestamp'] >= window_start) & 
                    (stim_frames['timestamp'] <= window_end)
                ]
                
                # Check if this fixation window contains any outliers
                if len(filtered_stim_frames) > 0:
                    if filtered_stim_frames['is_outlier'].sum() > 0:
                        outlier_fixation_windows += 1

fixation_window_outlier_pct = (outlier_fixation_windows / total_fixation_windows) * 100 if total_fixation_windows > 0 else 0

print("="*60)
print("OUTLIER ANALYSIS")
print("="*60)
print(f"\n--- FRAME-LEVEL STATISTICS ---")
print(f"Total frames with stimulus: {total_frames_with_stimulus}")
print(f"Outlier frames: {outlier_frames}")
print(f"Valid frames: {valid_frames_count}")
print(f"Outlier percentage: {outlier_percentage:.2f}%")

print(f"\n--- FIXATION WINDOW STATISTICS ---")
print(f"Total fixation windows: {total_fixation_windows}")
print(f"Fixation windows with outliers: {outlier_fixation_windows}")
print(f"Fixation windows without outliers: {total_fixation_windows - outlier_fixation_windows}")
print(f"Fixation window outlier percentage: {fixation_window_outlier_pct:.2f}%")

# Per subject analysis
print("\n" + "-"*60)
print("OUTLIER PERCENTAGE BY SUBJECT (Frame-level)")
print("-"*60)

subject_outlier_stats = []
for subject_id in frames_with_stimulus['subject_id'].unique():
    subject_data = frames_with_stimulus[frames_with_stimulus['subject_id'] == subject_id]
    total = len(subject_data)
    outliers = len(subject_data[subject_data['is_outlier'] == 1])
    pct = (outliers / total) * 100 if total > 0 else 0
    
    subject_outlier_stats.append({
        'subject_id': subject_id,
        'total_frames': total,
        'outlier_frames': outliers,
        'outlier_percentage': pct
    })
    
    print(f"{subject_id}: {outliers}/{total} ({pct:.2f}%)")




In [ ]:
# Create bar plot showing outlier percentage by horizontal stimulus position

# Calculate outlier percentage per horizontal position
outlier_by_horizontal = frames_with_stimulus.groupby('fixation_positionX_deg').agg({
    'is_outlier': ['sum', 'count']
}).reset_index()

outlier_by_horizontal.columns = ['fixation_x_deg', 'outlier_count', 'total_count']
outlier_by_horizontal['outlier_percentage'] = (outlier_by_horizontal['outlier_count'] / outlier_by_horizontal['total_count']) * 100

# Sort by horizontal position
outlier_by_horizontal = outlier_by_horizontal.sort_values('fixation_x_deg')

# Create figure
fig_outlier_by_horizontal, ax = plt.subplots(figsize=(14, 6))

# Create bar plot
bars = ax.bar(outlier_by_horizontal['fixation_x_deg'], 
              outlier_by_horizontal['outlier_percentage'],
              width=2)

# Add count annotations on top of bars
for idx, row in outlier_by_horizontal.iterrows():
    if row['outlier_percentage'] > 0:
        ax.text(row['fixation_x_deg'], 
                row['outlier_percentage'] + 1, 
                f"{row['outlier_percentage']:.1f}%",
                ha='center', 
                va='bottom',
                fontsize=12)

ax.set_xlabel('Horizontal Stimulus Position (°)')
ax.set_ylabel('Outlier Percentage (%)')
ax.set_title('Outlier Percentage by Horizontal Stimulus Position', fontweight='bold')
ax.set_ylim(0, max(outlier_by_horizontal['outlier_percentage']) * 1.2 if len(outlier_by_horizontal) > 0 else 100)
ax.set_axisbelow(True)

# Set x-axis ticks to show all stimulus positions
ax.set_xticks(outlier_by_horizontal['fixation_x_deg'].values)
ax.set_xticklabels([f'{x:.0f}' for x in outlier_by_horizontal['fixation_x_deg'].values], rotation=45, ha='right')
ax.grid(False, axis='both')


plt.tight_layout()
fig_outlier_by_horizontal.savefig(output_folder / 'fig_outlier_by_horizontal_position.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Outlier by horizontal position plot saved to {output_folder / 'fig_outlier_by_horizontal_position.png'}")

In [ ]:
# Filter out frames which are marked as outliers
valid_frames = valid_frames[valid_frames['is_outlier'] == 0]
print(f"Valid frames after outlier removal: {len(valid_frames)}")

In [ ]:
# Filter to keep only data from 0.5s to 1.5s after the first occurrence of each stimulus
filtered_frames = []

for subject_id in valid_frames['subject_id'].unique():
    subject_data = valid_frames[valid_frames['subject_id'] == subject_id]
    
    for eye in ['left', 'right']:
        eye_data = subject_data[subject_data['eye'] == eye]
        
        # Get all unique stimulus IDs
        unique_stimuli = eye_data['stimulus_id'].dropna().unique()
        
        for stim_id in unique_stimuli:
            # Get all frames for this stimulus
            stim_frames = eye_data[eye_data['stimulus_id'] == stim_id].sort_values('timestamp')
            
            if len(stim_frames) > 0:
                # Find the first timestamp for this stimulus
                first_timestamp = stim_frames['timestamp'].iloc[0]
                
                # Define the time window: 0.5s to 1.5s after first occurrence
                window_start = first_timestamp + PRE_OFFSET_TIME
                window_end = first_timestamp + 1 + POST_OFFSET_TIME

                # Get frames within this time window
                filtered_stim_frames = stim_frames[
                    (stim_frames['timestamp'] >= window_start) & 
                    (stim_frames['timestamp'] <= window_end)
                ]
                
                if len(filtered_stim_frames) > 0:
                    filtered_frames.append(filtered_stim_frames)

# Combine filtered frames
if filtered_frames:
    valid_frames = pd.concat(filtered_frames, ignore_index=True)
    print(f"\nFiltered to {len(valid_frames)} frames (0.5s-1.5s after first stimulus occurrence)")
    unique_stim_count = valid_frames.groupby(['subject_id', 'eye'])['stimulus_id'].nunique()
    print(f"Average stimuli per subject/eye: {unique_stim_count.mean():.1f}")
    print(f"Average frames per stimulus: {len(valid_frames) / unique_stim_count.sum():.1f}")
else:
    print("No frames remaining after filtering!")


In [ ]:
# Calculate errors
valid_frames['error_x'] = valid_frames['gaze_angle_x'] - valid_frames['fixation_positionX_deg']
valid_frames['error_y'] = valid_frames['gaze_angle_y'] - valid_frames['fixation_positionY_deg']

# Calculate absolute errors
valid_frames['abs_error_x'] = np.abs(valid_frames['error_x'])
valid_frames['abs_error_y'] = np.abs(valid_frames['error_y'])

# Calculate Euclidean distance error
valid_frames['euclidean_error'] = np.sqrt(valid_frames['error_x']**2 + valid_frames['error_y']**2)


# Add screen region column
def assign_screen_region(x):
    if x < -22:
        return 'left'
    elif x > 22:
        return 'right'
    else:
        return 'middle'

valid_frames['screen_region'] = valid_frames['fixation_positionX_deg'].apply(assign_screen_region)

print(f"\nError and screen region columns added successfully")

# Accuracy Analaysis

In [ ]:
# Calculate statistics per fixation (stimulus) per eye per subject
from helper_functions.calculate_bcea import calculate_bcea
from helper_functions.calculate_rms_s2s import calculate_rms_s2s

fixations = []

# Minimum number of frames required for valid statistics
MIN_FRAMES = 3

for subject_id in valid_frames['subject_id'].unique():
    subject_data = valid_frames[valid_frames['subject_id'] == subject_id]
    for eye in ['left', 'right']:
        eye_data = subject_data[subject_data['eye'] == eye]
        
        # Group by stimulus_id (each fixation point)
        for stim_id in eye_data['stimulus_id'].dropna().unique():
            stim_data = eye_data[eye_data['stimulus_id'] == stim_id]
            
            # Filter out NaN values in gaze data
            stim_data_valid = stim_data.dropna(subset=['gaze_angle_x', 'gaze_angle_y'])
            
            # Skip if not enough valid frames for statistics
            if len(stim_data_valid) < MIN_FRAMES:
                continue
                
            if len(stim_data_valid) > 0:
                # Get fixation position (should be constant for this stimulus)
                fix_x = stim_data_valid['fixation_positionX_deg'].iloc[0]
                fix_y = stim_data_valid['fixation_positionY_deg'].iloc[0]
                
                # Calculate mean gaze angles
                mean_gaze_x = stim_data_valid['gaze_angle_x'].mean()
                mean_gaze_y = stim_data_valid['gaze_angle_y'].mean()

                # Calculate std and bcea
                gaze_array = np.array([stim_data_valid['gaze_angle_x'].values, stim_data_valid['gaze_angle_y'].values]).T
                # std_gaze_x = stim_data['gaze_angle_x'].std()
                # std_gaze_y = stim_data['gaze_angle_y'].std()
                bcea_gaze, std_gaze_x, std_gaze_y, rho = calculate_bcea(gaze_array)

                # Calculate s2sRMS
                RMSs2s_gaze_x, RMSs2s_gaze_y, RMSs2s_euclidean = calculate_rms_s2s(gaze_array)

                # Calculate MAE for this fixation
                mae_x = stim_data_valid['abs_error_x'].mean()
                mae_y = stim_data_valid['abs_error_y'].mean()
                mae_euclidean = stim_data_valid['euclidean_error'].mean()
                
                fixations.append({
                    'eye': eye,
                    'subject_id': subject_id,
                    'stimulus_id': int(stim_id),
                    'fixation_x_deg': fix_x,
                    'fixation_y_deg': fix_y,
                    'mean_gaze_x_deg': mean_gaze_x,
                    'mean_gaze_y_deg': mean_gaze_y,
                    'std_gaze_x_deg': std_gaze_x,
                    'std_gaze_y_deg': std_gaze_y,
                    'bcea_gaze_deg2': bcea_gaze,
                    'RMSs2s_gaze_x_deg': RMSs2s_gaze_x,
                    'RMSs2s_gaze_y_deg': RMSs2s_gaze_y,
                    'RMSs2s_euclidean_deg': RMSs2s_euclidean,
                    'MAE_horizontal_deg': mae_x,
                    'MAE_vertical_deg': mae_y,
                    'MAE_euclidean_deg': mae_euclidean,
                    'n_frames': len(stim_data_valid)
                })

fixation_df = pd.DataFrame(fixations)
# print(f"Statistics calculated for {len(fixation_df)} fixations")
# print(f"\nSample of statistics per fixation:")
# print(fixation_df.head(10).to_string(index=False))

In [ ]:
# Boxplots of MAE by horizontal stimulus position

# Create figure with subplots
fig_mae_by_position, axes = plt.subplots(3,1, figsize=(7, 10))

y_max = 15  # degrees
fontsize = 14
tick_fontsize = 12

# Horizontal MAE by horizontal stimulus position
ax = axes[0]
sns.boxplot(data=fixation_df, x='fixation_x_deg', y='MAE_horizontal_deg', hue='eye', ax=ax)
ax.set_xlabel('Horizontal Stimulus Position (°)', fontsize=fontsize)
ax.set_ylabel('Horizontal MAE (°)', fontsize=fontsize)
ax.set_ylim(0, y_max)
ax.set_yticks(np.arange(0, y_max+1, 2))
ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', rotation=45)
ax.tick_params(axis='both', labelsize=tick_fontsize)
ax.get_legend().remove()

# Vertical MAE by horizontal stimulus position
ax = axes[1]
sns.boxplot(data=fixation_df, x='fixation_x_deg', y='MAE_vertical_deg', hue='eye', ax=ax)
ax.set_xlabel('Horizontal Stimulus Position (°)', fontsize=fontsize)
ax.set_ylabel('Vertical MAE (°)', fontsize=fontsize)
ax.set_ylim(0, y_max)
ax.set_yticks(np.arange(0, y_max+1, 2))
ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', rotation=45)
ax.tick_params(axis='both', labelsize=tick_fontsize)
ax.get_legend().remove()

# Eucleadian MAE by horizontal stimulus position
ax = axes[2]
sns.boxplot(data=fixation_df, x='fixation_x_deg', y='MAE_euclidean_deg', hue='eye', ax=ax)
ax.set_xlabel('Horizontal Stimulus Position (°)', fontsize=fontsize)
ax.set_ylabel('Euclidean MAE (°)', fontsize=fontsize)
ax.set_ylim(0, y_max)
ax.set_yticks(np.arange(0, y_max+1, 2))
ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', rotation=45)
ax.tick_params(axis='both', labelsize=tick_fontsize)
ax.legend(title='Eye', fontsize=12)

plt.tight_layout()
fig_mae_by_position.savefig(output_folder / 'fig_mae_by_position.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Boxplots of precision metrics by horizontal stimulus position

# Create figure with subplots for SD
fig_precision_by_position, axes = plt.subplots(2, 3, figsize=(14, 10))

y_max_std = 4.5  # degrees
y_max_bcea = 40  # degrees²
y_max_rms = 5.5  # degrees

# === SD METRICS ===
# Horizontal SD by horizontal stimulus position
ax = axes[0, 0]
sns.boxplot(data=fixation_df, x='fixation_x_deg', y='std_gaze_x_deg', hue='eye', ax=ax)
ax.set_xlabel('Horizontal Stimulus Position (°)', fontsize=6)
ax.set_ylabel('SD (°)', fontsize=12)
ax.set_ylim(0, y_max_std)
ax.set_title('Horizontal SD', fontsize=13, fontweight='bold')
ax.set_yticks(np.arange(0, y_max_std+0.5, 0.5))
ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', rotation=90)
ax.get_legend().remove()

# Vertical SD by horizontal stimulus position
ax = axes[0, 1]
sns.boxplot(data=fixation_df, x='fixation_x_deg', y='std_gaze_y_deg', hue='eye', ax=ax)
ax.set_xlabel('Horizontal Stimulus Position (°)', fontsize=6)
ax.set_ylabel('SD (°)', fontsize=12)
ax.set_ylim(0, y_max_std)
ax.set_title('Vertical SD', fontsize=13, fontweight='bold')
ax.set_yticks(np.arange(0, y_max_std+0.5, 0.5))
ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', rotation=90)
ax.get_legend().remove()

# BCEA by horizontal stimulus position
ax = axes[0, 2]
sns.boxplot(data=fixation_df, x='fixation_x_deg', y='bcea_gaze_deg2', hue='eye', ax=ax)
ax.set_xlabel('Horizontal Stimulus Position (°)', fontsize=6)
ax.set_ylabel('BCEA (°²)', fontsize=12)
ax.set_ylim(0, y_max_bcea)
ax.set_title('BCEA', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', rotation=90)
ax.legend(title='Eye', fontsize=12, loc='upper center')

# === RMS-S2S METRICS ===
# Horizontal RMS-S2S by horizontal stimulus position
ax = axes[1, 0]
sns.boxplot(data=fixation_df, x='fixation_x_deg', y='RMSs2s_gaze_x_deg', hue='eye', ax=ax)
ax.set_xlabel('Horizontal Stimulus Position (°)', fontsize=6)
ax.set_ylabel('RMS-S2S (°)', fontsize=12)
ax.set_ylim(0, y_max_rms)
ax.set_title('Horizontal RMS-S2S', fontsize=13, fontweight='bold')
ax.set_yticks(np.arange(0, y_max_rms+0.5, 0.5))
ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', rotation=90)
ax.get_legend().remove()

# Vertical RMS-S2S by horizontal stimulus position
ax = axes[1, 1]
sns.boxplot(data=fixation_df, x='fixation_x_deg', y='RMSs2s_gaze_y_deg', hue='eye', ax=ax)
ax.set_xlabel('Horizontal Stimulus Position (°)', fontsize=6)
ax.set_ylabel('RMS-S2S (°)', fontsize=12)
ax.set_ylim(0, y_max_rms)
ax.set_title('Vertical RMS-S2S', fontsize=13, fontweight='bold')
ax.set_yticks(np.arange(0, y_max_rms+0.5, 0.5))
ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', rotation=90)
ax.get_legend().remove()

# Euclidean RMS-S2S by horizontal stimulus position
ax = axes[1, 2]
sns.boxplot(data=fixation_df, x='fixation_x_deg', y='RMSs2s_euclidean_deg', hue='eye', ax=ax)
ax.set_xlabel('Horizontal Stimulus Position (°)', fontsize=6)
ax.set_ylabel('RMS-S2S (°)', fontsize=12)
ax.set_ylim(0, y_max_rms)
ax.set_title('Euclidean RMS-S2S', fontsize=13, fontweight='bold')
ax.set_yticks(np.arange(0, y_max_rms+0.5, 0.5))
ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', rotation=90)
ax.get_legend().remove()

plt.tight_layout()
fig_precision_by_position.savefig(output_folder / 'fig_precision_by_position.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate combined (Euclidean) metrics for fixations < 40 degrees
fixation_40deg_df = fixation_df[np.abs(fixation_df['fixation_x_deg']) < 40]

print("=== METRICS FOR FIXATIONS < 40° ===")
# print(fixation_40deg_df.keys())

# MAE metrics
print(f"\nAccuracy (MAE):")
print(f"  Horizontal MAE: {fixation_40deg_df['MAE_horizontal_deg'].mean():.3f}° ± {fixation_40deg_df['MAE_horizontal_deg'].std():.3f}°")
print(f"  Vertical MAE:   {fixation_40deg_df['MAE_vertical_deg'].mean():.3f}° ± {fixation_40deg_df['MAE_vertical_deg'].std():.3f}°")
print(f"  Euclidean MAE:  {fixation_40deg_df['MAE_euclidean_deg'].mean():.3f}° ± {fixation_40deg_df['MAE_euclidean_deg'].std():.3f}°")
# Combine horizontal and vertical MAE values into one series
combined_mae_values = pd.concat([
    fixation_40deg_df['MAE_horizontal_deg'], 
    fixation_40deg_df['MAE_vertical_deg']
])

print(f"  Combined MAE (H+V): {combined_mae_values.mean():.3f}° ± {combined_mae_values.std():.3f}°")

# STD metrics (precision)
print(f"\nPrecision (STD):")
print(f"  Horizontal STD: {fixation_40deg_df['std_gaze_x_deg'].mean():.3f}° ± {fixation_40deg_df['std_gaze_x_deg'].std():.3f}°")
print(f"  Vertical STD:   {fixation_40deg_df['std_gaze_y_deg'].mean():.3f}° ± {fixation_40deg_df['std_gaze_y_deg'].std():.3f}°")
# Same for STD
combined_std_values = pd.concat([
    fixation_40deg_df['std_gaze_x_deg'], 
    fixation_40deg_df['std_gaze_y_deg']
])
print(f"  Combined STD (H+V): {combined_std_values.mean():.3f}° ± {combined_std_values.std():.3f}°")


# RMSs2s metrics (precision)
print(f"\nPrecision (RMSs2s):")
print(f"  Horizontal RMSs2s: {fixation_40deg_df['RMSs2s_gaze_x_deg'].mean():.3f}° ± {fixation_40deg_df['RMSs2s_gaze_x_deg'].std():.3f}°")
print(f"  Vertical RMSs2s:   {fixation_40deg_df['RMSs2s_gaze_y_deg'].mean():.3f}° ± {fixation_40deg_df['RMSs2s_gaze_y_deg'].std():.3f}°")
# Same for RMSs2s
combined_RMSs2s_values = pd.concat([
    fixation_40deg_df['RMSs2s_gaze_x_deg'], 
    fixation_40deg_df['RMSs2s_gaze_y_deg']
])
print(f"  Combined RMSs2s (H+V): {combined_RMSs2s_values.mean():.3f}° ± {combined_RMSs2s_values.std():.3f}°")




In [ ]:
fixation_45deg_df = fixation_df[np.abs(fixation_df['fixation_x_deg']) > 40]

print(fixation_45deg_df.keys())
print(f"  Horizontal MAE: {fixation_45deg_df['MAE_horizontal_deg'].mean():.3f}° ± {fixation_45deg_df['MAE_horizontal_deg'].std():.3f}°")
print(f"  Vertical MAE:   {fixation_45deg_df['MAE_vertical_deg'].mean():.3f}° ± {fixation_45deg_df['MAE_vertical_deg'].std():.3f}°")
print(f"  Euclidean MAE:   {fixation_45deg_df['MAE_euclidean_deg'].mean():.3f}° ± {fixation_45deg_df['MAE_euclidean_deg'].std():.3f}°")
print(f"  Horizontal std: {fixation_45deg_df['std_gaze_x_deg'].mean():.3f}° ± {fixation_45deg_df['std_gaze_x_deg'].std():.3f}°")
print(f"  Vertical std:   {fixation_45deg_df['std_gaze_y_deg'].mean():.3f}° ± {fixation_45deg_df['std_gaze_y_deg'].std():.3f}°")

In [ ]:
# Calculate average MAE per subject and eye over all fixations
# make distinction between left(x<-22deg), middle(-22deg<x<22deg) and right(x>22deg) screen and over all screens

# First, add screen region to fixation_df
def assign_screen_region(x):
    if x < -22:
        return 'left'
    elif x > 22:
        return 'right'
    else:
        return 'middle'

fixation_df['screen_region'] = fixation_df['fixation_x_deg'].apply(assign_screen_region)

subject_eye_results = []

for subject_id in fixation_df['subject_id'].unique():
    subject_data = fixation_df[fixation_df['subject_id'] == subject_id]
    for eye in ['left', 'right']:
        eye_data = subject_data[subject_data['eye'] == eye]
        
        # Overall (all screens) - average the MAE and precision metrics across all fixations
        if len(eye_data) > 0:
            # Accuracy metrics
            mae_x = eye_data['MAE_horizontal_deg'].mean()
            mae_y = eye_data['MAE_vertical_deg'].mean()
            mae_euclidean = eye_data['MAE_euclidean_deg'].mean()
            
            # Precision metrics
            std_x = eye_data['std_gaze_x_deg'].mean()
            std_y = eye_data['std_gaze_y_deg'].mean()
            bcea = eye_data['bcea_gaze_deg2'].mean()
            rms_s2s_x = eye_data['RMSs2s_gaze_x_deg'].mean()
            rms_s2s_y = eye_data['RMSs2s_gaze_y_deg'].mean()
            rms_s2s_euclidean = eye_data['RMSs2s_euclidean_deg'].mean()
            
            subject_eye_results.append({
                'subject_id': subject_id,
                'eye': eye,
                'screen_region': 'all',
                'MAE_horizontal_deg': mae_x,
                'MAE_vertical_deg': mae_y,
                'MAE_euclidean_deg': mae_euclidean,
                'STD_horizontal_deg': std_x,
                'STD_vertical_deg': std_y,
                'BCEA_deg2': bcea,
                'RMS_S2S_horizontal_deg': rms_s2s_x,
                'RMS_S2S_vertical_deg': rms_s2s_y,
                'RMS_S2S_euclidean_deg': rms_s2s_euclidean,
                'n_fixations': len(eye_data)
            })
        
        # By screen region - average the MAE and precision metrics across fixations in each region
        for region in ['left', 'middle', 'right']:
            region_data = eye_data[eye_data['screen_region'] == region]
            if len(region_data) > 0:
                # Accuracy metrics
                mae_x = region_data['MAE_horizontal_deg'].mean()
                mae_y = region_data['MAE_vertical_deg'].mean()
                mae_euclidean = region_data['MAE_euclidean_deg'].mean()
                
                # Precision metrics
                std_x = region_data['std_gaze_x_deg'].mean()
                std_y = region_data['std_gaze_y_deg'].mean()
                bcea = region_data['bcea_gaze_deg2'].mean()
                rms_s2s_x = region_data['RMSs2s_gaze_x_deg'].mean()
                rms_s2s_y = region_data['RMSs2s_gaze_y_deg'].mean()
                rms_s2s_euclidean = region_data['RMSs2s_euclidean_deg'].mean()

                subject_eye_results.append({
                    'subject_id': subject_id,
                    'eye': eye,
                    'screen_region': region,
                    'MAE_horizontal_deg': mae_x,
                    'MAE_vertical_deg': mae_y,
                    'MAE_euclidean_deg': mae_euclidean,
                    'STD_horizontal_deg': std_x,
                    'STD_vertical_deg': std_y,
                    'BCEA_deg2': bcea,
                    'RMS_S2S_horizontal_deg': rms_s2s_x,
                    'RMS_S2S_vertical_deg': rms_s2s_y,
                    'RMS_S2S_euclidean_deg': rms_s2s_euclidean,
                    'n_fixations': len(region_data)
                })

mae_df = pd.DataFrame(subject_eye_results)
# print("MAE and Precision metrics per subject, eye, and screen region (averaged across fixations):")
# print(mae_df.to_string(index=False))



# Bias Analysis (Systematic Errors)

In [ ]:
# Calculate signed error for each fixation to identify directional bias

# Add signed error columns to fixation_df
fixation_df['signed_error_x'] = fixation_df['mean_gaze_x_deg'] - fixation_df['fixation_x_deg']
fixation_df['signed_error_y'] = fixation_df['mean_gaze_y_deg'] - fixation_df['fixation_y_deg']


In [ ]:
# Error direction and magnitude across the screen both eyes combined
# Calculate mean signed error per fixation position (averaged across subjects)

fig_error_vectors_comb, ax = plt.subplots(1, 1, figsize=(6, 4))

# Calculate mean signed error per fixation position across all subjects
mean_errors = fixation_df.groupby(['fixation_x_deg', 'fixation_y_deg']).agg({
    'signed_error_x': 'mean',
    'signed_error_y': 'mean'
}).reset_index()

# Plot fixation points as crosses
ax.scatter(mean_errors['fixation_x_deg'], mean_errors['fixation_y_deg'],
            c='black', marker='+', s=50, linewidths=1, zorder=3, label='Fixation targets')

# Plot error vectors as quiver plot
quiver = ax.quiver(
    mean_errors['fixation_x_deg'], 
    mean_errors['fixation_y_deg'],
    mean_errors['signed_error_x'],
    mean_errors['signed_error_y'],
    color='red',
    alpha=0.7,
    scale=50,  # Adjust to control arrow length
    width=0.004,
    zorder=2
)

ax.set_xlabel('Horizontal Position (°)')
ax.set_ylabel('Vertical Position (°)')
ax.set_xlim(-65, 65)
ax.set_ylim(-25, 25)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

plt.tight_layout()
fig_error_vectors_comb.savefig(output_folder / 'fig_error_vectors_combined.png', dpi=300, bbox_inches='tight')
plt.show()

# Single Subject Analysis

In [ ]:
# Single subject trace plot - camera usage, light source usage, and horizontal gaze

# Select subject and eye to plot
SUBJECT_TO_PLOT = 'emr02'
EYE_TO_PLOT = 'left'

import matplotlib.pyplot as plt

# Set default font size for all plot elements
plt.rcParams['font.size'] = 16  # Base font size
plt.rcParams['axes.titlesize'] = 16  # Titles
plt.rcParams['axes.labelsize'] = 16  # Axis labels
plt.rcParams['xtick.labelsize'] = 16  # X-axis tick labels
plt.rcParams['ytick.labelsize'] = 16  # Y-axis tick labels
plt.rcParams['legend.fontsize'] = 16  # Legend

# Parse camera list from string - extract only numbers
def parse_cameras(cam_string):
    if pd.isna(cam_string):
        return []
    if isinstance(cam_string, str):
        cam_string = cam_string.replace('[', '').replace(']', '').replace("'", '').replace('"', '').strip()
        if cam_string:
            cameras = [cam.strip() for cam in cam_string.split(',')]
            # Extract only the numeric part: 'cam1' -> 1, 'cam2' -> 2
            return [int(cam.replace('cam', '')) for cam in cameras if cam.startswith('cam')]
    return []

# Parse light source list from string - extract only numbers
def parse_sources(source_string):
    if pd.isna(source_string):
        return []
    if isinstance(source_string, str):
        source_string = source_string.replace('[', '').replace(']', '').replace("'", '').replace('"', '').strip()
        if source_string:
            sources = [source.strip() for source in source_string.split(',')]
            # Extract only the numeric part and filter out '0'
            return [int(src) for src in sources if src.isdigit() and src != '0']
    return []

# Filter data for selected subject and eye
subject_data = combined_df[
    (combined_df['subject_id'] == SUBJECT_TO_PLOT) & 
    (combined_df['eye'] == EYE_TO_PLOT)
].copy().sort_values('timestamp')

if len(subject_data) == 0:
    print(f"No data found for subject {SUBJECT_TO_PLOT}, {EYE_TO_PLOT} eye")
else:
    # Normalize timestamps to start at 0
    first_timestamp = subject_data['timestamp'].iloc[0]
    subject_data['timestamp'] = subject_data['timestamp'] - first_timestamp
    
    # Parse camera and light source lists
    subject_data['camera_list'] = subject_data['cameras'].apply(parse_cameras)
    subject_data['light_source_list'] = subject_data['light_sources'].apply(parse_sources)
    
    
    # Create figure with 3 subplots (only horizontal gaze)
    fig, axes = plt.subplots(3, 1, figsize=(16, 10), sharex=True)
    
    eye_data = subject_data.copy()
    
    if len(eye_data) > 0:
        # Use timestamps instead of frame numbers
        timestamps = eye_data['timestamp'].values
        
        ax_cam = axes[0]
        ax_ls = axes[1]
        ax_h = axes[2]
        
        # ===== PLOT CAMERA TIMELINE =====
        y_pos = 0
        camera_numbers = sorted([int(k.split('_')[1]) for k in CAMERA_COLORS.keys()])
        
        for cam_num in camera_numbers:
            segments = []
            segment_start = None
            
            for idx, row in eye_data.iterrows():
                time = row['timestamp']  # Use timestamp instead of frame_nr
                cameras_used = row['camera_list']
                
                if cam_num in cameras_used:
                    if segment_start is None:
                        segment_start = time
                else:
                    if segment_start is not None:
                        segments.append((segment_start, time))
                        segment_start = None
            
            if segment_start is not None:
                segments.append((segment_start, timestamps[-1]))
            
            for start, end in segments:
                ax_cam.barh(y_pos, end - start, left=start, height=0.8,
                            color=CAMERA_COLORS[f'cam_{cam_num}'], alpha=0.8, edgecolor='none')
            
            y_pos += 1
        
        ax_cam.set_yticks(range(len(camera_numbers)))
        ax_cam.set_yticklabels([f'C{num+1}' for num in camera_numbers])
        ax_cam.set_title(f'Camera Usage', fontsize=12, fontweight='bold')
        ax_cam.grid(True, alpha=0.2, axis='x')
        
        # ===== PLOT LIGHT SOURCE TIMELINE =====
        used_light_sources = set()
        for light_sources in eye_data['light_source_list']:
            used_light_sources.update(light_sources)

        available_light_sources = sorted([int(k.split('_')[1]) for k in LIGHT_SOURCE_COLORS.keys() if int(k.split('_')[1]) in used_light_sources])

        y_pos = 0
        for ls_num in available_light_sources:
            segments = []
            segment_start = None
            
            for idx, row in eye_data.iterrows():
                time = row['timestamp']  # Use timestamp instead of frame_nr
                light_sources_used = row['light_source_list']
                
                if ls_num in light_sources_used:
                    if segment_start is None:
                        segment_start = time
                else:
                    if segment_start is not None:
                        segments.append((segment_start, time))
                        segment_start = None
            
            if segment_start is not None:
                segments.append((segment_start, timestamps[-1]))
            
            for start, end in segments:
                ax_ls.barh(y_pos, end - start, left=start, height=0.8,
                            color=LIGHT_SOURCE_COLORS[f'light_{ls_num}'], alpha=0.8, edgecolor='none')
            
            y_pos += 1
        
        ax_ls.set_yticks(range(len(available_light_sources)))
        ax_ls.set_yticklabels([f'L{num}' for num in available_light_sources])
        ax_ls.set_title(f'Light Source Usage', fontsize=12, fontweight='bold')
        ax_ls.grid(True, alpha=0.2, axis='x')
        
        # ===== PLOT HORIZONTAL GAZE ANGLE =====
        valid_mask = eye_data['gaze_angle_x'].notna()
        segments = []
        segment_start = None
        
        for idx, is_valid in enumerate(valid_mask.values):
            if is_valid:
                if segment_start is None:
                    segment_start = idx
            else:
                if segment_start is not None:
                    segments.append((segment_start, idx))
                    segment_start = None
        
        if segment_start is not None:
            segments.append((segment_start, len(valid_mask)))
        
        for start, end in segments:
            segment_data = eye_data.iloc[start:end]
            ax_h.plot(segment_data['timestamp'], segment_data['gaze_angle_x'],  # Use timestamp
                        'b-', linewidth=1.5, label='Measured Gaze' if start == segments[0][0] else '')
        
        # Plot stimulus fixation positions (horizontal) as horizontal bars
        if 'fixation_positionX_deg' in eye_data.columns and 'stimulus_id' in eye_data.columns:
            stim_segments = []
            current_stim = None
            segment_start_time = None
            segment_position = None
            
            for idx, row in eye_data.iterrows():
                stim_id = row['stimulus_id']
                time = row['timestamp']  # Use timestamp instead of frame_nr
                position = row['fixation_positionX_deg']
                
                if pd.notna(stim_id):
                    if current_stim != stim_id:
                        if current_stim is not None:
                            stim_segments.append((segment_start_time, time, segment_position))
                        current_stim = stim_id
                        segment_start_time = time
                        segment_position = position
                else:
                    if current_stim is not None:
                        stim_segments.append((segment_start_time, time, segment_position))
                        current_stim = None
            
            if current_stim is not None:
                stim_segments.append((segment_start_time, timestamps[-1], segment_position))
            
            # Draw horizontal bars for each stimulus segment
            for i, (start_time, end_time, position) in enumerate(stim_segments):
                ax_h.hlines(position, start_time, end_time, 
                            colors='gray', linewidth=2, alpha=0.8,
                            label='Target Position' if i == 0 else '')
            
            # Highlight analysis windows
            if 'timestamp' in eye_data.columns:
                analysis_window_shown_inliers = False
                analysis_window_shown_outliers = False
                for stim_id in eye_data['stimulus_id'].dropna().unique():
                    stim_data = eye_data[eye_data['stimulus_id'] == stim_id].sort_values('timestamp')
                    
                    if len(stim_data) > 0 and stim_data['timestamp'].notna().any():
                        first_timestamp = stim_data['timestamp'].iloc[0]
                        window_start_time = first_timestamp + PRE_OFFSET_TIME
                        window_end_time = first_timestamp + 1 + POST_OFFSET_TIME
                        
                        window_data = stim_data[
                            (stim_data['timestamp'] >= window_start_time) & 
                            (stim_data['timestamp'] <= window_end_time)
                        ]
                        
                        if len(window_data) > 0:
                            if 1 in window_data['is_outlier'].values:
                                color = 'red'
                                label = 'Outliers' if not analysis_window_shown_outliers else ''
                                analysis_window_shown_outliers = True
                            else:
                                color = 'green'
                                label = 'Inliers' if not analysis_window_shown_inliers else ''
                                analysis_window_shown_inliers = True
                            
                            # Shade using timestamps directly
                            ax_h.axvspan(window_start_time, window_end_time, 
                                    alpha=0.15, color=color, label=label)
        
        ax_h.set_xlabel('Time (s)')  # Update label
        ax_h.set_ylabel('Horizontal Gaze Angle (°)')
        ax_h.set_title('Horizontal Gaze', fontsize=12, fontweight='bold')
        ax_h.grid(True, alpha=0.3)
        ax_h.set_ylim(-60, 60)
        ax_h.legend(loc='upper right')

    plt.tight_layout()
    fig.savefig(output_folder / f'fig_single_subject_{SUBJECT_TO_PLOT}_{EYE_TO_PLOT}.png', 
                dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n✓ Single subject trace plot created for {SUBJECT_TO_PLOT}, {EYE_TO_PLOT} eye")
    print(f"  Total frames: {len(eye_data)}")